# LSTM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import itertools
import os

# File path and naming convention
directory = '/Users/zxgan/FYP_Kubernetes/Dataset/'
file_prefix = 'node_node_'
file_suffix = '_dataset.csv'
num_files = 50

# Hyperparameter configurations
param_grid_ae = {
    'units': [32, 64, 128],
    'dropout_rate': [0.3,0.4, 0.5],
    'learning_rate': [0.0001, 0.001, 0.01],
    'batch_size': [16, 32, 64],
    'l2_reg': [0.001, 0.01, 0.1],
    'epochs': [100]
}
config_combinations = list(itertools.product(*param_grid_ae.values()))
param_grid_keys = list(param_grid_ae.keys())

# Initialize summary
results_summary = []

# Function to build Bidirectional LSTM model
def build_bidirectional_lstm(input_shape, num_classes, units, dropout_rate, l2_reg=None):
    reg = l2(l2_reg) if l2_reg else None
    model = Sequential([
        Bidirectional(LSTM(units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), input_shape=input_shape),
        Bidirectional(LSTM(units // 2, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)),
        Dense(units // 4, activation='relu', kernel_regularizer=reg),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Loop through all datasets
for i in range(num_files):
    file_path = f"{directory}{file_prefix}{i}{file_suffix}"
    print(f"Processing file: {file_path}")
    
    # Load and preprocess dataset
    data = pd.read_csv(file_path)
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    features = data.drop(columns=['timestamp', 'node_name', 'pod_status_Running', 'pod_status_Pending', 
                                   'pod_status_Succeeded', 'pod_status_Failed', 'pod_status_Unknown'])
    targets = data[['pod_status_Running', 'pod_status_Pending', 'pod_status_Succeeded', 
                    'pod_status_Failed', 'pod_status_Unknown']].idxmax(axis=1).astype('category').cat.codes
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42, stratify=targets)
    
    # Normalize the features
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Reshape inputs for LSTM
    X_train_lstm = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
    X_test_lstm = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])
    
    input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])
    num_classes = len(np.unique(y_train))
    
    # Train and evaluate models
    best_model = None
    best_config = None
    best_val_accuracy = 0

    for config in config_combinations:
        config_dict = dict(zip(param_grid_keys, config))
        print(f"Node {i} - Training with config: {config_dict}")
        
        # Build and compile model
        model = build_bidirectional_lstm(input_shape, num_classes, units=config_dict['units'], 
                                         dropout_rate=config_dict['dropout_rate'], l2_reg=config_dict['l2_reg'])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config_dict['learning_rate']),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        # Early stopping and learning rate scheduler
        early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
        
        # Train model
        history = model.fit(
            X_train_lstm, y_train,
            epochs=config_dict['epochs'],
            batch_size=config_dict['batch_size'],
            validation_data=(X_test_lstm, y_test),
            callbacks=[early_stopping, lr_scheduler],
            verbose=2
        )
        
        val_accuracy = max(history.history['val_accuracy'])
        
        # If better, save the model and configuration
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = model
            best_config = config_dict
    
    print(f"Node {i} - Best Configuration: {best_config}")
    print(f"Node {i} - Best Validation Accuracy: {best_val_accuracy:.4f}")
    
    # Save the best model for this dataset
    model_path = f"/Users/zxgan/FYP_Kubernetes/LSTM_Model/LSTM_Model_Node_{i}.h5"
    best_model.save(model_path)
    print(f"Best model saved for Node {i} at: {model_path}")
    
    # Add results to the summary
    results_summary.append({
        "Node": f"Node_{i}",
        "Best Config": str(best_config),
        "Best Validation Accuracy": best_val_accuracy
    })

# Create a DataFrame for the results
results_df = pd.DataFrame(results_summary)

# Save results to a CSV
summary_path = f"{directory}lstm_accuracy_summary.csv"
results_df.to_csv(summary_path, index=False)
print(f"Summary saved to: {summary_path}")

Processing file: /Users/zxgan/FYP_Kubernetes/Dataset/node_node_0_dataset.csv
Node 0 - Training with config: {'units': 64, 'dropout_rate': 0.4, 'learning_rate': 0.0001, 'batch_size': 16, 'l2_reg': 0.001, 'epochs': 100}
Epoch 1/100
17/17 - 2s - loss: 1.6350 - accuracy: 0.1955 - val_loss: 1.6349 - val_accuracy: 0.1791 - lr: 1.0000e-04 - 2s/epoch - 140ms/step
Epoch 2/100
17/17 - 0s - loss: 1.6350 - accuracy: 0.1729 - val_loss: 1.6346 - val_accuracy: 0.1940 - lr: 1.0000e-04 - 41ms/epoch - 2ms/step
Epoch 3/100
17/17 - 0s - loss: 1.6346 - accuracy: 0.1955 - val_loss: 1.6343 - val_accuracy: 0.2239 - lr: 1.0000e-04 - 41ms/epoch - 2ms/step
Epoch 4/100
17/17 - 0s - loss: 1.6351 - accuracy: 0.2030 - val_loss: 1.6340 - val_accuracy: 0.2239 - lr: 1.0000e-04 - 41ms/epoch - 2ms/step
Epoch 5/100
17/17 - 0s - loss: 1.6330 - accuracy: 0.2143 - val_loss: 1.6338 - val_accuracy: 0.2239 - lr: 1.0000e-04 - 42ms/epoch - 2ms/step
Epoch 6/100
17/17 - 0s - loss: 1.6333 - accuracy: 0.2030 - val_loss: 1.6335 - val_

Epoch 1/100
9/9 - 2s - loss: 1.6366 - accuracy: 0.2180 - val_loss: 1.6345 - val_accuracy: 0.2090 - lr: 1.0000e-04 - 2s/epoch - 259ms/step
Epoch 2/100
9/9 - 0s - loss: 1.6353 - accuracy: 0.2143 - val_loss: 1.6344 - val_accuracy: 0.1940 - lr: 1.0000e-04 - 26ms/epoch - 3ms/step
Epoch 3/100
9/9 - 0s - loss: 1.6348 - accuracy: 0.2180 - val_loss: 1.6341 - val_accuracy: 0.2090 - lr: 1.0000e-04 - 25ms/epoch - 3ms/step
Epoch 4/100
9/9 - 0s - loss: 1.6333 - accuracy: 0.2406 - val_loss: 1.6338 - val_accuracy: 0.2239 - lr: 1.0000e-04 - 24ms/epoch - 3ms/step
Epoch 5/100
9/9 - 0s - loss: 1.6349 - accuracy: 0.2368 - val_loss: 1.6337 - val_accuracy: 0.2239 - lr: 1.0000e-04 - 25ms/epoch - 3ms/step
Epoch 6/100
9/9 - 0s - loss: 1.6350 - accuracy: 0.2143 - val_loss: 1.6336 - val_accuracy: 0.2239 - lr: 1.0000e-04 - 25ms/epoch - 3ms/step
Epoch 7/100
9/9 - 0s - loss: 1.6340 - accuracy: 0.2030 - val_loss: 1.6334 - val_accuracy: 0.2239 - lr: 1.0000e-04 - 25ms/epoch - 3ms/step
Epoch 8/100
9/9 - 0s - loss: 1.634

Epoch 1/100
17/17 - 2s - loss: 1.6601 - accuracy: 0.1842 - val_loss: 1.6596 - val_accuracy: 0.2537 - lr: 1.0000e-04 - 2s/epoch - 137ms/step
Epoch 2/100
17/17 - 0s - loss: 1.6586 - accuracy: 0.2218 - val_loss: 1.6589 - val_accuracy: 0.2537 - lr: 1.0000e-04 - 45ms/epoch - 3ms/step
Epoch 3/100
17/17 - 0s - loss: 1.6580 - accuracy: 0.2143 - val_loss: 1.6580 - val_accuracy: 0.2239 - lr: 1.0000e-04 - 43ms/epoch - 3ms/step
Epoch 4/100
17/17 - 0s - loss: 1.6560 - accuracy: 0.1955 - val_loss: 1.6572 - val_accuracy: 0.2687 - lr: 1.0000e-04 - 43ms/epoch - 3ms/step
Epoch 5/100
17/17 - 0s - loss: 1.6566 - accuracy: 0.1880 - val_loss: 1.6564 - val_accuracy: 0.2687 - lr: 1.0000e-04 - 43ms/epoch - 3ms/step
Epoch 6/100
17/17 - 0s - loss: 1.6552 - accuracy: 0.2030 - val_loss: 1.6555 - val_accuracy: 0.2537 - lr: 1.0000e-04 - 57ms/epoch - 3ms/step
Epoch 7/100
17/17 - 0s - loss: 1.6535 - accuracy: 0.2293 - val_loss: 1.6547 - val_accuracy: 0.2687 - lr: 1.0000e-04 - 65ms/epoch - 4ms/step
Epoch 8/100
17/17 - 

Epoch 1/100
9/9 - 2s - loss: 1.6593 - accuracy: 0.2030 - val_loss: 1.6593 - val_accuracy: 0.1940 - lr: 1.0000e-04 - 2s/epoch - 272ms/step
Epoch 2/100
9/9 - 0s - loss: 1.6582 - accuracy: 0.2368 - val_loss: 1.6589 - val_accuracy: 0.2090 - lr: 1.0000e-04 - 29ms/epoch - 3ms/step
Epoch 3/100
9/9 - 0s - loss: 1.6603 - accuracy: 0.2293 - val_loss: 1.6585 - val_accuracy: 0.1940 - lr: 1.0000e-04 - 26ms/epoch - 3ms/step
Epoch 4/100
9/9 - 0s - loss: 1.6569 - accuracy: 0.2481 - val_loss: 1.6581 - val_accuracy: 0.1940 - lr: 1.0000e-04 - 27ms/epoch - 3ms/step
Epoch 5/100
9/9 - 0s - loss: 1.6564 - accuracy: 0.2444 - val_loss: 1.6577 - val_accuracy: 0.2090 - lr: 1.0000e-04 - 26ms/epoch - 3ms/step
Epoch 6/100
9/9 - 0s - loss: 1.6566 - accuracy: 0.1880 - val_loss: 1.6573 - val_accuracy: 0.1791 - lr: 1.0000e-04 - 27ms/epoch - 3ms/step
Epoch 7/100
9/9 - 0s - loss: 1.6552 - accuracy: 0.2594 - val_loss: 1.6569 - val_accuracy: 0.1791 - lr: 1.0000e-04 - 27ms/epoch - 3ms/step
Epoch 8/100
9/9 - 0s - loss: 1.656

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

Epoch 1/100
14/14 - 3s - loss: 1.6344 - accuracy: 0.1943 - val_loss: 1.6342 - val_accuracy: 0.2075 - lr: 1.0000e-04 - 3s/epoch - 204ms/step
Epoch 2/100
14/14 - 0s - loss: 1.6355 - accuracy: 0.1469 - val_loss: 1.6339 - val_accuracy: 0.2642 - lr: 1.0000e-04 - 37ms/epoch - 3ms/step
Epoch 3/100
14/14 - 0s - loss: 1.6336 - accuracy: 0.1943 - val_loss: 1.6337 - val_accuracy: 0.2642 - lr: 1.0000e-04 - 38ms/epoch - 3ms/step
Epoch 4/100
14/14 - 0s - loss: 1.6338 - accuracy: 0.1991 - val_loss: 1.6334 - val_accuracy: 0.2830 - lr: 1.0000e-04 - 37ms/epoch - 3ms/step
Epoch 5/100
14/14 - 0s - loss: 1.6327 - accuracy: 0.2322 - val_loss: 1.6332 - val_accuracy: 0.2642 - lr: 1.0000e-04 - 37ms/epoch - 3ms/step
Epoch 6/100
14/14 - 0s - loss: 1.6323 - accuracy: 0.2275 - val_loss: 1.6329 - val_accuracy: 0.2830 - lr: 1.0000e-04 - 38ms/epoch - 3ms/step
Epoch 7/100
14/14 - 0s - loss: 1.6323 - accuracy: 0.2038 - val_loss: 1.6326 - val_accuracy: 0.3396 - lr: 1.0000e-04 - 41ms/epoch - 3ms/step
Epoch 8/100
14/14 - 

Epoch 1/100
7/7 - 3s - loss: 1.6359 - accuracy: 0.1706 - val_loss: 1.6351 - val_accuracy: 0.1698 - lr: 1.0000e-04 - 3s/epoch - 358ms/step
Epoch 2/100
7/7 - 0s - loss: 1.6348 - accuracy: 0.2180 - val_loss: 1.6349 - val_accuracy: 0.1887 - lr: 1.0000e-04 - 24ms/epoch - 3ms/step
Epoch 3/100
7/7 - 0s - loss: 1.6350 - accuracy: 0.1991 - val_loss: 1.6347 - val_accuracy: 0.1887 - lr: 1.0000e-04 - 24ms/epoch - 3ms/step
Epoch 4/100
7/7 - 0s - loss: 1.6325 - accuracy: 0.2180 - val_loss: 1.6345 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 24ms/epoch - 3ms/step
Epoch 5/100
7/7 - 0s - loss: 1.6359 - accuracy: 0.2038 - val_loss: 1.6344 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 24ms/epoch - 3ms/step
Epoch 6/100
7/7 - 0s - loss: 1.6335 - accuracy: 0.1991 - val_loss: 1.6342 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 24ms/epoch - 3ms/step
Epoch 7/100
7/7 - 0s - loss: 1.6359 - accuracy: 0.2133 - val_loss: 1.6340 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 24ms/epoch - 3ms/step
Epoch 8/100
7/7 - 0s - loss: 1.634

Epoch 1/100
14/14 - 3s - loss: 1.6614 - accuracy: 0.1611 - val_loss: 1.6594 - val_accuracy: 0.2453 - lr: 1.0000e-04 - 3s/epoch - 201ms/step
Epoch 2/100
14/14 - 0s - loss: 1.6592 - accuracy: 0.2085 - val_loss: 1.6586 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 42ms/epoch - 3ms/step
Epoch 3/100
14/14 - 0s - loss: 1.6591 - accuracy: 0.1706 - val_loss: 1.6580 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 43ms/epoch - 3ms/step
Epoch 4/100
14/14 - 0s - loss: 1.6571 - accuracy: 0.2417 - val_loss: 1.6574 - val_accuracy: 0.2453 - lr: 1.0000e-04 - 40ms/epoch - 3ms/step
Epoch 5/100
14/14 - 0s - loss: 1.6570 - accuracy: 0.2180 - val_loss: 1.6569 - val_accuracy: 0.2453 - lr: 1.0000e-04 - 44ms/epoch - 3ms/step
Epoch 6/100
14/14 - 0s - loss: 1.6555 - accuracy: 0.2512 - val_loss: 1.6563 - val_accuracy: 0.3019 - lr: 1.0000e-04 - 43ms/epoch - 3ms/step
Epoch 7/100
14/14 - 0s - loss: 1.6559 - accuracy: 0.2464 - val_loss: 1.6558 - val_accuracy: 0.2642 - lr: 1.0000e-04 - 54ms/epoch - 4ms/step
Epoch 8/100
14/14 - 

Epoch 1/100
7/7 - 3s - loss: 1.6599 - accuracy: 0.2559 - val_loss: 1.6589 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 3s/epoch - 365ms/step
Epoch 2/100
7/7 - 0s - loss: 1.6596 - accuracy: 0.2227 - val_loss: 1.6586 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 27ms/epoch - 4ms/step
Epoch 3/100
7/7 - 0s - loss: 1.6599 - accuracy: 0.1896 - val_loss: 1.6581 - val_accuracy: 0.2075 - lr: 1.0000e-04 - 27ms/epoch - 4ms/step
Epoch 4/100
7/7 - 0s - loss: 1.6567 - accuracy: 0.2559 - val_loss: 1.6576 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 24ms/epoch - 3ms/step
Epoch 5/100
7/7 - 0s - loss: 1.6570 - accuracy: 0.2227 - val_loss: 1.6571 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 25ms/epoch - 4ms/step
Epoch 6/100
7/7 - 0s - loss: 1.6567 - accuracy: 0.2180 - val_loss: 1.6566 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 27ms/epoch - 4ms/step
Epoch 7/100
7/7 - 0s - loss: 1.6567 - accuracy: 0.2417 - val_loss: 1.6561 - val_accuracy: 0.2264 - lr: 1.0000e-04 - 27ms/epoch - 4ms/step
Epoch 8/100
7/7 - 0s - loss: 1.656

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
